__future__ is needed for interactive widgets, has to be imported before everything else.

In [ ]:
from __future__ import print_function

In [ ]:
# default_exp ui

# cheviz ui

**FIXME:** There are compatibility problems with JupyterLab and ipywidgets, see for instance https://github.com/jupyter-widgets/ipywidgets/issues/2488 or https://stackoverflow.com/questions/49542417/how-to-get-ipywidgets-working-in-jupyter-lab – I need to address that at some point even though I am still using classic Jupyter notebooks. The mentioned workarounds advice to install JavaScript extensions to JupyterLab via npm, of all things. That sounds to me like the same bad decisions that affected the Gnome 3 desktop.

In [ ]:
#hide
from nbdev.showdoc import *

We can use interactive widgets with Jupyter for our minimal UI.

In [ ]:
#export
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from cheviz import core, data

In [ ]:
def fun(x:int):
    print(x)
    
interact(fun, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.fun(x:int)>

In [ ]:
games = list(data.games(data.fetch()['Candidates_1953.pgn']))
games_slider = widgets.IntSlider(min=0, max=len(games)-1, step=1, value=0)
widgets.interactive(lambda game_id: print(games[game_id]), game_id=games_slider)

interactive(children=(IntSlider(value=0, description='game_id', max=209), Output()), _dom_classes=('widget-int…

OK, perhaps we can build a simple chess game replay UI?

In [ ]:
#export
from typing import Callable
import chess

def showGameUi(games:list, games_id:int=0, move:int=0, echo:int=1):
    board = chess.Board()
    
    def gameUi(reduced:Callable, square_filter:Callable, game_id:int=0):
        curr = games[game_id]
        moves_list = list(curr[1])
        ms = data.makeMoveSequencer(moves_list)
        
        def applyMoves(move:int, echo:int):
            board.reset()
            
            for i in range(move):
                board.push(moves_list[i])
            
            display(board)
            if len(board.move_stack) > 0:
                adjusted_wm = move if not board.turn else move - 1
                adjusted_bm = move if board.turn else move - 1

                core.show(reduced(ms(range(adjusted_wm - echo, adjusted_wm), chess.WHITE, square_filter)))
                core.show(reduced(ms(range(adjusted_bm - echo, adjusted_bm), chess.BLACK, square_filter)))
                                
        moves_slider = widgets.IntSlider(min=0, max=len(moves_list), step=1, value=move)
        echo_slider = widgets.IntSlider(min=1, max=len(moves_list), step=2, value=echo)
        display(widgets.interactive(applyMoves, move=moves_slider, echo=echo_slider))
        print(curr[2])
        
    metrics_dd = widgets.Dropdown(options=[data.diffReduce, sum], value=data.diffReduce, description='Metric:', disabled=False)
    square_filter_dd = widgets.Dropdown(options=[core.attack, core.defence], value=core.attack, description='Square filter:', disabled=False)
    games_slider = widgets.IntSlider(min=0, max=len(games)-1, step=1, value=games_id)
    display(widgets.interactive(gameUi, reduced=metrics_dd, square_filter=square_filter_dd, game_id=games_slider))

For each UI element that controls input (here: our 3 sliders) we need a function to act as callback for the widget. Because I want those functions to implicitly share *some* state and yet not be visible as surface API of the ui module, I have chosen to nest those functions instead. Both showGameUi(...) and gameUi(...) act as closures. Since applyMoves(...) takes two arguments it receives input from two sliders (moves_slider, echo_slider). Analogous to applyMoves(...), gameUi(...) takes three arguments. The metric and square filter drop-down list both store/select function pointers.

A rather complex part of the code deals with a typical problem in chess: We have two sides, with alternating moves between them. At move 1, white made its first move and it's black's turn. Once black moves, white is an extra half move away from its initial position, without having moved itself. A move echo larger than 1 makes this issue clearly visible for instance. Our move selection mechanism (here: a Python range) needs to reflect that issue, hence the need for adjusted_{wm, bm} variables. One can discuss whether it might be cleaner to have two separate move stacks, one for each side, simply to avoid mis-steps wrt. move selection when in between moves.

In [ ]:
showGameUi(games, games_id=57, move=23, echo=5)

interactive(children=(Dropdown(description='Metric:', options=(<function diffReduce at 0x7efc59676d08>, <built…

We can override the CSS for IPython's display function so that we can show multiple elements next to each other.

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
    flex-wrap: wrap;
}
"""

HTML('<style>{}</style>'.format(CSS))